# Weighted K Means Clustering   


## 1. Introduction

The Weighted K-means clustering is a new k-means clustering algorithm that can automatically weight variables based on the importance of the variables in clustering. Weighted k-means adds a new step to the basic k-means algorithm to update the variable weights based on the current partition of data. It uses a weight calculation formula that minimizes the cost function of clustering given a fixed partition of data.    



 ## 2. Implementation on IRIS Dataset

Here I implement the Weighted K means clustering algorithm on the [IRIS dataset](https://archive.ics.uci.edu/ml/datasets/iris). The data set contains 3 classes of 50 instances each, where each class refers to a type of iris plant namely **Iris-setosa, Iris-versicolor** and **Iris-virginica**.   

I would use the original [paper](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.652.9691&rep=rep1&type=pdf) by Joshua Zhexue Huang, Michael K. Ng, Hongqiang Rong, and Zichen Li as a reference and almost the same set of notations for the variables.   

Let $X= \{ X_{1}, \ X_2,...,\ X_n \}$ be a set of $n$ objects. Object $X_i= \{ x_{i,1}, \ x_{i,2},...,\ x_{i,m}\}$ is characterized by a set of $m$ variables. We want to partition the $n$ objects into $k$ clusters that minimizes the objective function $P$ with unknown variables $U$,$Z$ and $W$ as follows: $$  P(U,Z,W)= \sum_{l=1}^{k}\sum_{i=1}^{n}\sum_{j=1}^{m} u_{i,l} w_j^{\beta} d(x_{i,j}, z_{l,j})  $$ where,  
- $U$ is an $n \times k$ partition matrix, $u_{i,l}$ is a binary variable, and $u_{i,l}=1$ indicates that object $i$ is allocated to cluster $l$;  
- $Z= \{ Z_{1}, \ Z_2,...,\ Z_k \}$ is a set of $k$ vectors representing the centroids of the $k$ clusters;   
- $d(x_{i,j}, z_{l,j}) $ is a distance measure between object $i$ and the centroid of the cluster $l$ on the $j$th variable. We define here $d(x_{i,j}, z_{l,j}) $ as : $$d(x_{i,j}, z_{l,j})= (x_{i,j}-z_{l,j})^{2}.$$   
- $W= \{ w_{1}, \ w_2,...,\ w_m \}$ be the weights for the $m$ variables such that $\sum_{j=1}^{m} w_{j}=1$ , $0 \le w_j \le 1.$   
So to minimize the cost function $P$, we have the Weighted k-means clustering as follows:  

**Weighted k-means Algorithm**    
1. Randomly choose an initial $Z^{0}=\{Z_{1},Z_{2},...,Z_{k}\}$ and randomly generate a set of initial weights $W^{0}=[ w_{1}^{0},\ w_{2}^{0},...,\ w_{m}^{0}] \ (\sum_{j=1}^{m} w_{j}=1)$. Determine $U^{0}$ such that $P(U^{0},Z^{0},W^{0})$ is minimized. Set $t=0$ ;   
2. Let $\hat{Z}=Z^{t}$ and $\hat{W}=W^{t}$, solve problem $P(U,\hat{Z},\hat{W})$ to obtain $U^{t+1}$. If $P(U^{t+1},\hat{Z},\hat{W})=P(U^{t},\hat{Z},\hat{W})$, output $(U^{t},\hat{Z},\hat{W})$ and stop; otherwise go to step 3;   
3. Let $\hat{U}=U^{t+1}$ and $\hat{W}=W^{t}$, solve problem $P(\hat{U},Z,\hat{W})$ to obtain $Z^{t+1}$. If $P(\hat{U},Z^{t+1},\hat{W})=P(\hat{U},Z^{t},\hat{W})$, output $(\hat{U},Z^{t},\hat{W})$ and stop; otherwise go to step 4;   
4. Let $\hat{U}=U^{t+1}$ and $\hat{Z}=Z^{t+1}$, solve problem $P(\hat{U},\hat{Z},W)$ to obtain $W^{t+1}$. If $P(\hat{U},\hat{Z},W^{t+1})=P(\hat{U},\hat{Z},W^{t})$, output $(\hat{U},\hat{Z},W^{t})$ and stop; otherwise, set $t=t+1$ go to step 2.






#### 2.1 Reading the dataset

We start by importing the necessary libraries (pandas and Numpy) and reading the dataset. I exclude the last column of our dataset (Species) and denote the new dataset (with just the first four columns) as $X$.

In [ ]:
pip install pandas

In [ ]:
import pandas as pd
import numpy as np
import math
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
iris=pd.read_csv ("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",
                  names = ["Sepal Length", "Sepal Width", "Petal Length", "Petal Width", "Class" ])  #Reading the csv file
X = iris.iloc[:,0:4]
y = iris.iloc[:,-1]
print(X)

#### 2.2 Defining a few distance functions

In [ ]:
def np_ed(point_1, point_2,A):
    distance = np.sum((np.dot(A,np.square(np.array(point_1) - np.array(point_2)))))
    return distance

def np_ed0(point_1, point_2):
    array_1, array_2 = np.array(point_1), np.array(point_2)
    distance0 = np.square(np.array(point_1)[0] - np.array(point_2)[0])
    return distance0

def np_ed1(point_1, point_2):
    array_1, array_2 = np.array(point_1), np.array(point_2)
    distance1 = np.square(np.array(point_1)[1] - np.array(point_2)[1])
    return distance1

def np_ed2(point_1, point_2):
    array_1, array_2 = np.array(point_1), np.array(point_2)
    distance2 = np.square(np.array(point_1)[2] - np.array(point_2)[2])
    return distance2

def np_ed3(point_1, point_2):
    array_1, array_2 = np.array(point_1), np.array(point_2)
    distance3 = np.square(np.array(point_1)[3] - np.array(point_2)[3])
    return distance3

#### 2.3 Initializing the first centroids, weights and determining $T$ by minimizing $P$

As the 1st step of the Weighted K Means clustering we initialize the centroid vector and randomly generate the set of initial weights. Then we determine the clusters for each data points in the first iteration.   
**Replacing $U$ by $T$**   
In the original paper, to denote the clusters for each data point in every iteration, the $n \times k$ matrix $U$ was used. The elements $u_{i,l}$ of $U$ is a binary variable, and $u_{i,l}=1$ indicates that object $i$ is allocated to cluster $l$. For brevity and ease, I use a different variable $T$ which is a $(150 \times 1)$ column vector, each element of $T$ is a number from the set $\{1,2,...,k\}$ and $t_{i,1}=l$ ,$(l \in \ \{1,2,...,k\})$, means that the $i$th data point belongs to the $l$th cluster. So rewriting our cost function $P$ in terms of $T$, $$  P(T,Z,W)= \sum_{l=1}^{k}\sum_{i=1}^{n}\sum_{j=1}^{m} \mathbb{1}(t_{i,1}=l) w_j^{\beta} d(x_{i,j}, z_{l,j})  $$ where $\mathbb{1}$ is the indicator function defined as 
$$\begin{equation}
\mathbb{1}(t_{i,1}=l) =
    \begin{cases}
        1 & \text{if the $i$th data point belongs to the $l$th cluster} \\
        0 & \text{otherwise}
    \end{cases}
\end{equation}$$

For the IRIS dataset, we have the no. of data points $n=150$, the number of clusters $k=3$ and the no. of variables $m=4$. So our cost function $P$ for the IRIS dataset is  $$  P(T,Z,W)= \sum_{l=1}^{3}\sum_{i=1}^{150}\sum_{j=0}^{3} \mathbb{1}(t_{i,1}=l) w_j^{\beta} d(x_{i,j}, z_{l,j})  $$


In [ ]:
M1=X.iloc[5]            # Initializing the centroids
M2=X.iloc[62]
M3=X.iloc[105]
Z=[M1,M2,M3]
B=2                      #Selecting Beta(B)
W=np.array([[pow(0.69,B),0,0,0],[0,pow(0.1,B),0,0],[0,0,pow(0.03,B),0], [0,0,0,pow(0.18,B)]])  #Initializing the weights

#### 2.4 Iteration for the new clusters ($T$- 2nd Step of W k-means)

This is the 2nd step of the W k-means clustering. Using the $Z$ and $W$ of the previous iteration we solve for the $T$ that minimizes the cost function $P$. The following code gives us the number of data points that belong to each cluster in this iteration and the value of $P$ with $Z$, $W$ and the new $T$.

In [ ]:
T=[]
P=0
for i in range(150):
    ed1 = np_ed(M1, X.iloc[i],W)
    ed2 = np_ed(M2, X.iloc[i],W)     # Calculating the new distances and creating the new clusters 
    ed3 = np_ed(M3, X.iloc[i],W)
    mini = min(ed1, ed2, ed3) 
    if (mini==ed1):
          S=1
    elif (mini==ed2):
         S=2
    else:
          S=3
    T=T+[S]    
np.T=np.array(T)    

for i in range(150):
    P =  P + np_ed(X.iloc[i], Z[np.T[i]-1],W)

    
printmd("**The Matrix T :**")
print(np.T)
printmd("**No. of data points in 1st cluster:**") 
print(np.count_nonzero(np.T == 1))
printmd("**No. of data points in 2nd cluster:**") 
print(np.count_nonzero(np.T == 2))
printmd("**No. of data points in 3rd cluster:**")
print(np.count_nonzero(np.T == 3))  
printmd("**Centroids:**")
print(Z)
printmd("**The Weight Matrix:**")
print(W)
printmd("**Value of P:**")
print(P)

#### 2.5 Iteration for the centroids ($Z$- 3rd Step of W k-means)

This is the 3rd step of the W k-means clustering. Using the $T$ and $W$ of the previous iteration we solve for the $Z$ that minimizes the cost function $P$. Solving for $Z$, we get that $P$ is minimized when $$z_{l,j}= \frac{\sum_{i=1}^{150} \mathbb{1}(t_{i,1}=l)\ x_{i,j}}{\sum_{i=1}^{150} \mathbb{1}(t_{i,1}=l)}$$where $1 \le l \le 3$ and $0 \le j \le 3$.  The following code gives us the new centroids and the value of $P$.

In [ ]:
Z1=[0,0,0,0]
Z2=[0,0,0,0]                    
Z3=[0,0,0,0]
P=0
for j in range(150):
    if (np.T[j]==1):
        Z1=(Z1+X.iloc[j])
    elif (np.T[j]==2):
        Z2=(Z2+X.iloc[j])                       #Calculating the new centroids
    else:
        Z3=(Z3+X.iloc[j])
M1=Z1/np.count_nonzero(np.T == 1)
M2=Z2/np.count_nonzero(np.T == 2)
M3=Z3/np.count_nonzero(np.T == 3)
Z=[M1,M2,M3]    

for i in range(150):
    P=  P + np_ed(X.iloc[i], Z[np.T[i]-1],W)

printmd("**The Matrix T :**")
print(np.T)
printmd("**No. of data points in 1st cluster:**") 
print(np.count_nonzero(np.T == 1))
printmd("**No. of data points in 2nd cluster:**") 
print(np.count_nonzero(np.T == 2))
printmd("**No. of data points in 3rd cluster:**")
print(np.count_nonzero(np.T == 3))  
printmd("**Centroids:**")
print(Z)
printmd("**The Weight Matrix:**")
print(W)
printmd("**Value of P:**")
print(P)    

#### 2.6 Iteration for the weights ($W$- 4th Step of W k-means)

This is the 4th step of the W k-means clustering. Using the $T$ and $Z$ of the previous iteration we solve for the $W$ that minimizes the cost function $P$. Solving for $W$, we get that $P$ is minimized when $$w_j= \frac{1}{\sum_{t=0}^{3} \left[\frac{D_j}{D_t}\right]^{\frac{1}{\beta - 1}}} \ \ \ \ \ (j\in \{0,1,2,3\} \ \text{and} \ \beta=2)$$ where $$D_t= \sum_{l=1}^{3}\sum_{i=1}^{150} \mathbb{1}(t_{i,1}=l) d(x_{i,t}, z_{l,t})  $$ The following code gives us the new weights and the value of $P$.

In [ ]:
D_0= 0
D_1= 0
D_2= 0
D_3= 0
P= 0
for i in range(150):
    D_0 = D_0 + np_ed0(X.iloc[i], Z[np.T[i]-1])
    D_1 = D_1 + np_ed1(X.iloc[i], Z[np.T[i]-1])         #Calculating the D_i's
    D_2 = D_2 + np_ed2(X.iloc[i], Z[np.T[i]-1])
    D_3 = D_3 + np_ed3(X.iloc[i], Z[np.T[i]-1])
    T2=  (1/(pow((D_0),1/(B-1))) + 1/(pow((D_1),1/(B-1))) + 1/(pow((D_2),1/(B-1))) + 1/(pow((D_3),1/(B-1))))


w0= 1/((pow((D_0),1/(B-1)))*(T2))
w1= 1/((pow((D_1),1/(B-1)))*(T2))
w2= 1/((pow((D_2),1/(B-1)))*(T2))
w3= 1/((pow((D_3),1/(B-1)))*(T2))

W=np.array([[pow(w0,B),0,0,0],[0,pow(w1,B),0,0],[0,0,pow(w2,B),0], [0,0,0,pow(w3,B)]])

for i in range(150):
    P= P + np_ed(X.iloc[i], Z[np.T[i]-1],W)

printmd("**The Matrix T :**")
print(np.T)
printmd("**No. of data points in 1st cluster:**") 
print(np.count_nonzero(np.T == 1))
printmd("**No. of data points in 2nd cluster:**") 
print(np.count_nonzero(np.T == 2))
printmd("**No. of data points in 3rd cluster:**")
print(np.count_nonzero(np.T == 3))  
printmd("**Centroids:**")
print(Z)
printmd("**The Weight Matrix:**")
print(W)
printmd("**Value of P:**")
print(P)